# Training Workflow

- Array-task to download videos,
- Array-task to break them down into frames,
- Array-task to run luminance algorithm to pick key frames.


In [1]:
from flytekit.sdk.tasks import python_task, inputs, outputs
from flytekit.sdk.types import Types
from flytekit.sdk.workflow import workflow_class, Output, Input
from workflows.classifier_evaluate_workflow import evaluate_lp
from workflows.classifier_train_workflow import train_lp, DEFAULT_VALIDATION_DATA_RATIO, \
    DEFAULT_TRAINING_VALIDATION_CONFIG_FILE
from workflows.data_preparation_workflow import data_prep
from workflows.driver_workflow import pick_second


@workflow_class
class ClassifierTrainWorkflow:
    available_streams_mpblobs = Input([Types.MultiPartBlob], required=True)
    available_streams_names = Input([Types.String], required=True)
    streams_metadata_path = Input(Types.String, required=True)
    training_validation_config_json = Input(Types.Generic, default=ujson.loads(open(DEFAULT_TRAINING_VALIDATION_CONFIG_FILE).read()))
    validation_data_ratio = Input(Types.Float, default=DEFAULT_VALIDATION_DATA_RATIO)

    rearrange_data_task = rearrange_data(
        available_streams_mpblobs=available_streams_mpblobs,
        available_streams_names=available_streams_names,
        training_validation_config_json=training_validation_config_json,
        streams_metadata_path=streams_metadata_path,
        validation_data_ratio=validation_data_ratio,
    )

    train_on_datasets_task = train_on_datasets(
        training_validation_config_json=training_validation_config_json,
        training_clean_mpblob=rearrange_data_task.outputs.training_clean_mpblob,
        training_dirty_mpblob=rearrange_data_task.outputs.training_dirty_mpblob,
        validation_clean_mpblob=rearrange_data_task.outputs.validation_clean_mpblob,
        validation_dirty_mpblob=rearrange_data_task.outputs.validation_dirty_mpblob,
    )

    trained_models = Output(train_on_datasets_task.outputs.model_blobs, sdk_type=[Types.Blob])
    model_file_names = Output(train_on_datasets_task.outputs.model_files_names, sdk_type=[Types.String])


train_lp = ClassifierTrainWorkflow.create_launch_plan()

Using TensorFlow backend.


NameError: name 'ujson' is not defined